In [1]:
import OpenDartReader
import pandas as pd

In [2]:
api_key = '92c176817e681dcc4ad263eb3fa5182792b0b7a3'
dart = OpenDartReader(api_key)

In [22]:
fs_2019 = dart.finstate_all(corp='005930', bsns_year='2020', fs_div='CFS', reprt_code=11011)
fs_2020_3Q = dart.finstate_all(corp='005930', bsns_year='2020', fs_div='CFS', reprt_code=11014)

In [4]:
int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['BS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].iat[0])

276136188000000

In [5]:
corp = '005930' # 기업코드
bsns_year = '2020' # 기준연도
fs_div = 'CFS' # 연결재무제표: CFS / 개별재무제표 :OFS
reprt_code = '11011' # 사업보고서: 11011 / 반기보고서: 11012 / 1분기보고서: 11013 /  3분기보고서: 11014 

def get_equity(corp, bsns_year, fs_div, reprt_code) :
    df = dart.finstate_all(corp=corp, bsns_year=bsns_year, fs_div=fs_div, reprt_code=reprt_code)
    equity = df.loc[df['sj_div'].isin(['BS']) & df['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount']
    # 재무상태표: 'sj_div'='BS' / 자본총계 : 'account_id'='ifrs-full_Equity' / 값 : 'thstrm_amount'
    equity = int(equity.iat[0])
    # df의 값 추출
    return equity

In [6]:
print(get_equity(corp, bsns_year, fs_div, reprt_code))

275948016000000


In [7]:
corp = '005930' # 기업코드
bsns_year = '2020' # 기준연도
fs_div = 'CFS' # 연결재무제표: CFS / 개별재무제표 :OFS
reprt_code = '11011' # 사업보고서: 11011 / 반기보고서: 11012 / 1분기보고서: 11013 /  3분기보고서: 11014 

In [8]:
def get_liability(corp, bsns_year, fs_div, reprt_code) :
    df = dart.finstate_all(corp=corp, bsns_year=bsns_year, fs_div=fs_div, reprt_code=reprt_code)
    liability = df.loc[df['sj_div'].isin(['BS']) & df['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount']
    # 재무상태표: 'sj_div'='BS' / 부채총계 : 'account_id'='ifrs-full_Liabilities' / 값 : 'thstrm_amount'
    liability = int(liability.iat[0])
    # df의 값 추출
    return liability

In [9]:
print(get_liability(corp, bsns_year, fs_div, reprt_code))

102287702000000


In [10]:
def get_profit_loss(corp, bsns_year, fs_div, reprt_code) :
    df = dart.finstate_all(corp=corp, bsns_year=bsns_year, fs_div=fs_div, reprt_code=reprt_code)
    profit_loss = df.loc[df['sj_div'].isin(['IS']) & df['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount']
    # 포괄손익계산서: 'sj_div'='IS' / 당기순이익 : 'account_id'='ifrs-full_Equity' / 값 : 'thstrm_amount'
    profit_loss = int(profit_loss.iat[0])
    # df의 값 추출
    return profit_loss

In [11]:
print(get_profit_loss(corp, bsns_year, fs_div, reprt_code))

26090846000000


In [12]:
def get_grossprofit(corp, bsns_year, fs_div, reprt_code) :
    df = dart.finstate_all(corp=corp, bsns_year=bsns_year, fs_div=fs_div, reprt_code=reprt_code)
    grossprofit = df.loc[df['sj_div'].isin(['IS']) & df['account_id'].isin(['ifrs-full_GrossProfit']), 'thstrm_amount']
    # 포괄손익계산서: 'sj_div'='IS' / 당기순이익 : 'account_id'='ifrs-full_Equity' / 값 : 'thstrm_amount'
    grossprofit = int(grossprofit.iat[0])
    # df의 값 추출
    return grossprofit

In [13]:
print(get_grossprofit(corp, bsns_year, fs_div, reprt_code))

92318692000000


In [14]:
from pykrx import stock

In [15]:
df = stock.get_market_cap_by_ticker("20200625")
df.loc['005930']['시가총액']

309831714345000

In [16]:
df = stock.get_market_fundamental_by_ticker("20210104", market="KOSDAQ")
df.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
060310,745,0.000000,3.029297,0,0.000000,0
054620,10530,0.000000,0.759766,0,0.000000,0
265520,7468,38.000000,3.410156,671,0.199951,50
211270,5479,163.375000,1.519531,51,0.839844,70
035760,134364,26.703125,1.059570,5310,0.990234,1400


In [17]:
tickers = stock.get_market_ticker_list("20190225", market="KOSPI")  # KOSPI / KOSDAQ / KONEX / ALL / default = KOSPI
print(tickers[:10]) 

['095570', '068400', '006840', '027410', '282330', '138930', '001460', '001465', '001040', '079160']


In [30]:
temp_dic = {'tickers':[], 'quality':[]}
for ticker in tickers :
    try :
        corp = ticker # 기업코드
        bsns_year = '2020' # 기준연도
        fs_div = 'CFS' # 연결재무제표: CFS / 개별재무제표 :OFS
        reprt_code = '11011' # 사업보고서: 11011 / 반기보고서: 11012 / 1분기보고서: 11013 /  3분기보고서: 11014 
        grossprofit = get_grossprofit(corp, bsns_year, fs_div, reprt_code)
        equity = get_equity(corp, bsns_year, fs_div, reprt_code)
        liability = get_liability(corp, bsns_year, fs_div, reprt_code)
        quality = grossprofit / (equity + liability)
        temp_dic['tickers'].append(ticker)
        temp_dic['quality'].append(quality)
    except :
        pass

In [40]:
pd.set_option('display.max.rows', 500)
data=pd.DataFrame(temp_dic)
data['rank'] = data['quality'].rank(ascending=False)
data.sort_values(by='rank')

,tickers,quality,rank
58,053210,0.759906,1.0
13,051900,0.717729,2.0
61,031430,0.648247,3.0
14,032640,0.568780,4.0
75,185750,0.516879,5.0
23,009450,0.449669,6.0
73,007570,0.413805,7.0
22,005610,0.349025,8.0
16,066570,0.338496,9.0
76,001630,0.328893,10.0


In [72]:
corp = '095570' # 기업코드
bsns_year = '2019' # 기준연도
fs_div = 'CFS' # 연결재무제표: CFS / 개별재무제표 :OFS
reprt_code = '11011' # 사업보고서: 11011 / 반기보고서: 11012 / 1분기보고서: 11013 /  3분기보고서: 11014 
df = dart.finstate_all(corp=corp, bsns_year=bsns_year, fs_div=fs_div, reprt_code=reprt_code)
profit_loss = df.loc[df['sj_div'].isin(['CIS']) & df['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount']
profit_loss

74    45969636300
Name: thstrm_amount, dtype: object